In [6]:
import sys
import json
import pickle
import pyspark
from functools import reduce
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import udf, mean, lit, stddev, col, expr, when
from pyspark.sql.types import DoubleType, ArrayType, ShortType, LongType, IntegerType
import pandas as pd
from collections import OrderedDict
from datetime import date
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt

from pyspark.ml import Pipeline, PipelineModel

from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import DecisionTreeClassifier

from pyspark.ml.feature import StringIndexer, VectorAssembler, VectorIndexer
from pyspark.ml import PipelineModel

In [1]:
sc = SparkContext.getOrCreate()
sql = SQLContext.getOrCreate(sc)

In [3]:
STORAGE_ACCOUNT_SUFFIX = 'core.windows.net'
STORAGE_ACCOUNT_NAME = os.environ['STORAGE_ACCOUNT_NAME']

wasbUrlOutput = "wasb://{0}@{1}.blob.{2}/features.parquet".format(
            'intermediate',
            STORAGE_ACCOUNT_NAME,
            STORAGE_ACCOUNT_SUFFIX)

dfa = spark.read.parquet(wasbUrlOutput)
dfa.printSchema()

root
 |-- EnqueuedTimeUtc: string (nullable = true)
 |-- machineID: string (nullable = true)
 |-- speed: double (nullable = true)
 |-- temperature: double (nullable = true)
 |-- pressure: double (nullable = true)
 |-- f0: double (nullable = true)
 |-- f1: double (nullable = true)
 |-- f2: double (nullable = true)
 |-- a0: double (nullable = true)
 |-- a1: double (nullable = true)
 |-- a2: double (nullable = true)
 |-- temperature_n: double (nullable = true)
 |-- pressure_n: double (nullable = true)
 |-- f0_n: double (nullable = true)
 |-- f1_n: double (nullable = true)
 |-- f2_n: double (nullable = true)
 |-- a0_n: double (nullable = true)
 |-- a1_n: double (nullable = true)
 |-- a2_n: double (nullable = true)



In [7]:
features = [c for c in dfa.columns if c not in ['machineID', 'EnqueuedTimeUtc']]

# assemble features
va = VectorAssembler(inputCols=(features), outputCol='features')

# this is a hack!! In the current simulated dataset, there are 4 machines: machine 0 is "good," 
# whereas the rest are experiencing different failures. This will eventually be produced by merging 
# telemetry with the "maintenance log"
feat_data = va.transform(dfa).withColumn('label_e', dfa.machineID.substr(-1, 1).cast(IntegerType()))
dfa.unpersist()
feat_data.persist(pyspark.StorageLevel.DISK_ONLY)
feat_data.limit(10).toPandas().head()

,EnqueuedTimeUtc,machineID,speed,temperature,pressure,f0,f1,f2,a0,a1,...,temperature_n,pressure_n,f0_n,f1_n,f2_n,a0_n,a1_n,a2_n,features,label_e
0,2018-03-13T23:30:50.2920000Z,pm1-354,1143.7500,20.258794,1693.614698,37.0,73.0,75.0,1044.404476,724.869558,...,0.017713,1.480756,0.032350,0.063825,0.065574,0.913141,0.633766,0.626888,"[1143.75, 20.2587943809, 1693.61469839, 37.0, ...",4
1,2018-03-13T23:30:50.4480000Z,pm1-355,1246.8750,20.304156,1757.185113,40.0,20.0,81.0,1010.888092,756.150383,...,0.016284,1.409271,0.032080,0.016040,0.064962,0.810737,0.606436,0.544514,"[1246.875, 20.3041562235, 1757.1851125, 40.0, ...",5
2,2018-03-13T23:30:51.3550000Z,pm1-354,1181.8750,20.355704,1817.808709,39.0,97.0,78.0,1166.979098,972.222250,...,0.017223,1.538072,0.032998,0.082073,0.065997,0.987396,0.822610,0.760270,"[1181.875, 20.3557035837, 1817.80870908, 39.0,...",4
3,2018-03-13T23:30:51.4480000Z,pm1-355,1288.4375,20.417235,1898.551986,42.0,85.0,84.0,1122.998787,856.292329,...,0.015847,1.473531,0.032598,0.065971,0.065195,0.871597,0.664597,0.654390,"[1288.4375, 20.417235309, 1898.55198592, 42.0,...",5
4,2018-03-13T23:30:52.4170000Z,pm1-354,1200.9375,20.455311,1915.679343,238.0,40.0,99.0,1295.409910,1128.891514,...,0.017033,1.595153,0.198179,0.033307,0.082436,1.078666,0.940009,0.874146,"[1200.9375, 20.4553109108, 1915.67934282, 238....",4


In [8]:
# set maxCategories so features with > 10 distinct values are treated as continuous.
featureIndexer = VectorIndexer(inputCol="features", 
                               outputCol="indexedFeatures", 
                               maxCategories=10).fit(feat_data)

# fit on whole dataset to include all labels in index
labelIndexer = StringIndexer(inputCol="label_e", outputCol="indexedLabel").fit(feat_data)

In [9]:
training, test = feat_data.randomSplit([0.8, 0.2], seed=12345)
print(training.count())
print(test.count())
feat_data.limit(5).toPandas().head()

15417
3943


,EnqueuedTimeUtc,machineID,speed,temperature,pressure,f0,f1,f2,a0,a1,...,temperature_n,pressure_n,f0_n,f1_n,f2_n,a0_n,a1_n,a2_n,features,label_e
0,2018-03-13T23:30:50.2920000Z,pm1-354,1143.7500,20.258794,1693.614698,37.0,73.0,75.0,1044.404476,724.869558,...,0.017713,1.480756,0.032350,0.063825,0.065574,0.913141,0.633766,0.626888,"[1143.75, 20.2587943809, 1693.61469839, 37.0, ...",4
1,2018-03-13T23:30:50.4480000Z,pm1-355,1246.8750,20.304156,1757.185113,40.0,20.0,81.0,1010.888092,756.150383,...,0.016284,1.409271,0.032080,0.016040,0.064962,0.810737,0.606436,0.544514,"[1246.875, 20.3041562235, 1757.1851125, 40.0, ...",5
2,2018-03-13T23:30:51.3550000Z,pm1-354,1181.8750,20.355704,1817.808709,39.0,97.0,78.0,1166.979098,972.222250,...,0.017223,1.538072,0.032998,0.082073,0.065997,0.987396,0.822610,0.760270,"[1181.875, 20.3557035837, 1817.80870908, 39.0,...",4
3,2018-03-13T23:30:51.4480000Z,pm1-355,1288.4375,20.417235,1898.551986,42.0,85.0,84.0,1122.998787,856.292329,...,0.015847,1.473531,0.032598,0.065971,0.065195,0.871597,0.664597,0.654390,"[1288.4375, 20.417235309, 1898.55198592, 42.0,...",5
4,2018-03-13T23:30:52.4170000Z,pm1-354,1200.9375,20.455311,1915.679343,238.0,40.0,99.0,1295.409910,1128.891514,...,0.017033,1.595153,0.198179,0.033307,0.082436,1.078666,0.940009,0.874146,"[1200.9375, 20.4553109108, 1915.67934282, 238....",4


In [10]:
model_type = 'RandomForest' # Use 'DecisionTree', or 'RandomForest'

# train a model.
if model_type == 'DecisionTree':
    model = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures",
                                      # Maximum depth of the tree. (>= 0) 
                                      # E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'
                                      maxDepth=15,
                                      # Max number of bins for discretizing continuous features. 
                                      # Must be >=2 and >= number of categories for any categorical feature.
                                      maxBins=32, 
                                      # Minimum number of instances each child must have after split. 
                                      # If a split causes the left or right child to have fewer than 
                                      # minInstancesPerNode, the split will be discarded as invalid. Should be >= 1.
                                      minInstancesPerNode=1, 
                                      # Minimum information gain for a split to be considered at a tree node.
                                      minInfoGain=0.0, 
                                      # Criterion used for information gain calculation (case-insensitive). 
                                      # Supported options: entropy, gini')
                                      impurity="gini")

    ##=======================================================================================================================
    #elif model_type == 'GBTClassifier':
    #    cls_mthd = GBTClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")
    ##=======================================================================================================================
else:    
    model = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", 
                                      # Passed to DecisionTreeClassifier
                                      maxDepth=15, 
                                      maxBins=32, 
                                      minInstancesPerNode=1, 
                                      minInfoGain=0.0,
                                      impurity="gini",
                                      # Number of trees to train (>= 1)
                                      numTrees=50, 
                                      # The number of features to consider for splits at each tree node. 
                                      # Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n].
                                      featureSubsetStrategy="sqrt", 
                                      # Fraction of the training data used for learning each 
                                      # decision tree, in range (0, 1].' 
                                      subsamplingRate = 0.632)

# chain indexers and model in a Pipeline
pipeline_cls_mthd = Pipeline(stages=[labelIndexer, featureIndexer, model])

# train model.  This also runs the indexers.
model_pipeline = pipeline_cls_mthd.fit(training)

In [11]:
# make predictions. The Pipeline does all the same operations on the test data
predictions = model_pipeline.transform(test)

# Create the confusion matrix for the multiclass prediction results
# This result assumes a decision boundary of p = 0.5
conf_table = predictions.stat.crosstab('indexedLabel', 'prediction')
confuse = conf_table.toPandas().sort_values(by=['indexedLabel_prediction'])
confuse.head()

,indexedLabel_prediction,0.0,1.0,2.0,3.0
3,0.0,989,0,0,0
1,1.0,0,1001,0,0
0,2.0,0,0,994,0
2,3.0,0,0,0,959
